# Pyranges for selecting regions by overlap

In [87]:
import pyranges

In [88]:
import sys
# sys.path.append('../../src')

In [89]:
from features.nucleotide import BEDSeqLoader

In [90]:
bsl = BEDSeqLoader(bed_path='../../data/processed/GRCh38/221111_128bp_minoverlap64_mincov2_nc10_tissues/regions.bed.gz',
                   ref_path='/dhc/dsets/reference_genomes/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly_renamed.fa')

In [91]:
bsl

In [92]:
bsl.bed.head()

,Chromosome,Start,End,Name,Score
0,1,10054,10182,c1,0
1,1,10182,10310,c2,0
2,1,10310,10438,c3,0
3,1,10438,10566,c4,0
4,1,10566,10694,c5,0


In [93]:
bsl.bed["Chromosome"].describe()

count     15996490
unique          23
top              1
freq       1329379
Name: Chromosome, dtype: object

In [94]:
bsl.bed['i'] = bsl.bed.index.values # "i" will contain the original index preserving the ordering of the BED-file

In [95]:
bed = pyranges.PyRanges(df=bsl.bed, copy_df=False) # we create a pyranges object from the DataFrame

In [96]:
bsl.bed.i.values

array([       0,        1,        2, ..., 15996487, 15996488, 15996489])

In [97]:
len(bsl.bed.i.values)

15996490

In [98]:
bed.df.i.values

array([      0,       1,       2, ..., 8182882, 8182883, 8182884])

In [99]:
len(bed.df.i.values)

15996490

In [100]:
type(bed.df.i.values)

numpy.ndarray

In [101]:
all(bed.df.i.values == bsl.bed.i.values) # note that pyranges internally switches the ordering around .... this makes it a pain to link to the labels in the LabelLoaderHDF5 

False

In [102]:
all(bed.i == bsl.bed.i.values) # also happens when accessed with __getattr__ 

False

In [103]:
bed.i

0                0
1                1
2                2
3                3
4                4
            ...   
8182880    8182880
8182881    8182881
8182882    8182882
8182883    8182883
8182884    8182884
Name: i, Length: 15996490, dtype: int64

In [104]:
print(bed) # the column "i" in the pyranges contains the original positional index in the BED-file

+--------------+-----------+-----------+------------+-----------+-----------+
| Chromosome   | Start     | End       | Name       | Score     | i         |
| (category)   | (int32)   | (int32)   | (object)   | (int64)   | (int64)   |
|--------------+-----------+-----------+------------+-----------+-----------|
| 1            | 10054     | 10182     | c1         | 0         | 0         |
| 1            | 10182     | 10310     | c2         | 0         | 1         |
| 1            | 10310     | 10438     | c3         | 0         | 2         |
| 1            | 10438     | 10566     | c4         | 0         | 3         |
| ...          | ...       | ...       | ...        | ...       | ...       |
| X            | 156030363 | 156030491 | c7432941   | 0         | 8182881   |
| X            | 156030491 | 156030619 | c7432942   | 0         | 8182882   |
| X            | 156030619 | 156030747 | c7432943   | 0         | 8182883   |
| X            | 156030747 | 156030875 | c7432944   | 0         

In [105]:
# we define a set of query ranges:
query_ranges = pyranges.from_dict({"Chromosome": [1, 2, 3, 'X'], "Start": [0, 0, 0, 0], "End": [1000000, 1000000, 1000000, 1000000]})

In [106]:
# this could be an easy way to get i
idx_overlap = bed.overlap(query_ranges).i.values

In [107]:
bed.overlap(query_ranges)

,Chromosome,Start,End,Name,Score,i
0,1,10054,10182,c1,0,0
1,1,10182,10310,c2,0,1
2,1,10310,10438,c3,0,2
3,1,10438,10566,c4,0,3
4,1,10566,10694,c5,0,4
...,...,...,...,...,...,...
20606,X,999139,999267,c6599872,0,7288720
20607,X,999267,999395,c6599873,0,7288721
20608,X,999395,999523,c6599874,0,7288722
20609,X,999523,999651,c6599875,0,7288723


In [114]:
# returns the ones it touches!!!
test_query_ranges = pyranges.from_dict({"Chromosome": [1], "Start": [10060], "End": [10183]})
print(test_query_ranges)
print(type(test_query_ranges))
print()
print(type(pyranges.pyranges.PyRanges))
print(isinstance(test_query_ranges, pyranges.pyranges.PyRanges))
print()
test_idx_overlap = bed.overlap(test_query_ranges).i.values
print(test_idx_overlap)
print(bsl.get_seq(test_idx_overlap))
print(len(bsl.get_seq(test_idx_overlap)[0]))
print(len(bsl.get_seq(test_idx_overlap)[1]))

+--------------+-----------+-----------+
|   Chromosome |     Start |       End |
|   (category) |   (int32) |   (int32) |
|--------------+-----------+-----------|
|            1 |     10060 |     10183 |
+--------------+-----------+-----------+
Unstranded PyRanges object has 1 rows and 3 columns from 1 chromosomes.
For printing, the PyRanges was sorted on Chromosome.
<class 'pyranges.pyranges.PyRanges'>

<class 'type'>
True

[0 1]
['TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAA'
 'CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAACCCCA']
128
128


In [60]:
idx_overlap

array([      0,       1,       2, ..., 7288722, 7288723, 7288724])

In [61]:
len(idx_overlap)

20611

In [63]:
type(idx_overlap)

numpy.ndarray

In [62]:
bsl.get_seq(idx_overlap)

array(['TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAA',
       'CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAAACCCTAAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAACCCCA',
       'ACCCCAACCCCAACCCTAACCCCTAACCCTAACCCTAACCCTACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA',
       ...,
       'CCCTGTCTCTGCTAAAAATACAAATATCAGCCGAGAGTAGTGGTGGGCGTCTGTAGTCACAGCTACTTGGGAGGCTGAGACAGGAGAATCCCTTGAATCCGGGAGGCAGAGGTTGCAGTGAGCGGAGA',
       'TGATGCCATTGCACTCCGGTCTGGGCCACAGAGCGAGACTGTCAAAACAAAACGGAACAAAAAGAATGCTTTAAGAAGCGGGCGGATCACAAGATCCAGAGATTGAGAATAGCCTGGCTAACACAGTG',
       'AAACTCCGTCACTACTAAAAATACAAAAAAATTAGCCGGGCGTGGTGGCAGGTGCATGTAGTCTCAGCTCCTCAGGAGGCTGAGGCAGGAGAATGGTGTGAACCCAGGAGGCGGAGCTTGCAGTGAGC'],
      dtype='<U128')

In [64]:
type(bsl.get_seq(idx_overlap))

numpy.ndarray

In [65]:
len(bsl.get_seq(idx_overlap))

20611

In [66]:
len(bsl.get_seq(idx_overlap)[0])

128

In [67]:
bsl.bed.loc[idx_overlap] # this returns the correct ranges based on "i", i.e., the position in the BED file

,Chromosome,Start,End,Name,Score,i
0,1,10054,10182,c1,0,0
1,1,10182,10310,c2,0,1
2,1,10310,10438,c3,0,2
3,1,10438,10566,c4,0,3
4,1,10566,10694,c5,0,4
...,...,...,...,...,...,...
7288720,X,999139,999267,c6599872,0,7288720
7288721,X,999267,999395,c6599873,0,7288721
7288722,X,999395,999523,c6599874,0,7288722
7288723,X,999523,999651,c6599875,0,7288723


In [79]:
bed.df.loc[idx_overlap] # note that this returns the wrong ranges! PyRanges.index =/= position in BED file ("i") ........ 

,Chromosome,Start,End,Name,Score,i
0,1,10054,10182,c1,0,0
1,1,10182,10310,c2,0,1
2,1,10310,10438,c3,0,2
3,1,10438,10566,c4,0,3
4,1,10566,10694,c5,0,4
...,...,...,...,...,...,...
7288720,8,989704,989832,c7438225,0,8188742
7288721,8,989832,989960,c7438226,0,8188743
7288722,8,989960,990088,c7438227,0,8188744
7288723,8,990088,990216,c7438228,0,8188745


In [80]:
bed_idx_from_i = {k: v for k, v in zip(bed.df.i.values,bed.df.index.values)} # dictionary that maps "i" to position in the PyRanges

In [82]:
# bed_idx_from_i just the identity!

In [83]:
bed.df.loc[[*map(bed_idx_from_i.get, idx_overlap)]] # this actually returns the correct regions, but is way slower... (?) -> don't do this

,Chromosome,Start,End,Name,Score,i
0,1,10054,10182,c1,0,0
1,1,10182,10310,c2,0,1
2,1,10310,10438,c3,0,2
3,1,10438,10566,c4,0,3
4,1,10566,10694,c5,0,4
...,...,...,...,...,...,...
15102325,X,999139,999267,c6599872,0,7288720
15102326,X,999267,999395,c6599873,0,7288721
15102327,X,999395,999523,c6599874,0,7288722
15102328,X,999523,999651,c6599875,0,7288723


In [84]:
bsl.bed.loc[idx_overlap]

,Chromosome,Start,End,Name,Score,i
0,1,10054,10182,c1,0,0
1,1,10182,10310,c2,0,1
2,1,10310,10438,c3,0,2
3,1,10438,10566,c4,0,3
4,1,10566,10694,c5,0,4
...,...,...,...,...,...,...
7288720,X,999139,999267,c6599872,0,7288720
7288721,X,999267,999395,c6599873,0,7288721
7288722,X,999395,999523,c6599874,0,7288722
7288723,X,999523,999651,c6599875,0,7288723


In [85]:
# conclusion: we can get the index "i" of the overlapping regions using pyranges. I would not replace BEDSeqLoader.bed with a PyRanges-object though, because it messes with the ordering

In [86]:
del bsl, bed

# Subsetting based on labels

In [ ]:
from features.label import LabelLoaderHDF5

In [ ]:
lbl = LabelLoaderHDF5('data/processed/GRCh38/toydata/overlaps.h5', 'data/processed/GRCh38/toydata/regions.bed')

In [ ]:
lbl.get_labels(list(range(0,10))) # subsetting the data on this level is not easy to do efficiently

In [ ]:
lbl.get_labels_one_hot(list(range(0,10))).shape # subsetting is easier here (can just be by position in the second dimention)

In [ ]:
# IDEA: subsetting by the label id could be easier during initialization

In [ ]:
import h5py
import numpy as np

In [ ]:
f = h5py.File('data/processed/GRCh38/221111_128bp_minoverlap64_mincov2_nc10_tissues/overlaps.h5', 'r') # this is the full dataset

In [ ]:
l = len(f['query'])

In [ ]:
l

In [ ]:
q = f['query']

In [ ]:
# HDF5 data is stored in chunks. We can quickly iterate over chunks...

In [ ]:
# example: get every region with at least X labels:

In [ ]:
proc_chunk = []

for c in q.iter_chunks():
    
    # iterate over the HDF5 file in chunks
    
    chunk = q[c]
    chunk = np.stack(np.unique(chunk, return_counts=True), axis=1)
    
    if len(proc_chunk):
        if[chunk[0,0] == proc_chunk[-1][-1,0]]:
            proc_chunk[-1][-1,1] += chunk[0,1]
        chunk = chunk[1:]
    
    proc_chunk.append(chunk)

In [ ]:
proc_chunk = np.concatenate(proc_chunk)

In [ ]:
proc_chunk.shape

In [ ]:
len(np.unique(proc_chunk[:,0])) # the reason this is less long than the BED-file is that the ranges with 0 readouts are not stored in the HDF5

In [ ]:
proc_chunk[0:20] # [ i,  number of labels]

In [ ]:
# getting the regions with at least N experimental readouts is now trivial:

In [ ]:
min_cover = 5
idx = proc_chunk[proc_chunk[:,1] > min_cover,0]

In [ ]:
len(idx)

In [ ]:
del proc_chunk, chunk

In [ ]:
# second example: subset on a list of target labels

In [ ]:
t = f['target']

In [ ]:
# we can use a similar approach to filter for regions that have a specific set of labels:

In [ ]:
list(t.attrs.keys()) # 'N' contains the number of regions with a readout for each experiment, 'labels' contains the identifiers

In [ ]:
n_labels = len(t.attrs.get('labels'))

In [ ]:
n_labels

In [ ]:
labels_of_interest = np.random.choice(np.arange(n_labels), 100, replace=False)

In [ ]:
labels_of_interest

In [ ]:
import tqdm

In [ ]:
n_chunk = len(q) // q.chunks[0]

In [ ]:
n_chunk

In [ ]:
proc_chunk = []

i = 1

with tqdm.tqdm(total=n_chunk) as pbar:
    
    # this is a bit slower, having a progress bar makes it more satisfying

    for c in q.iter_chunks():
        
        # iterate over the HDF5 file in chunks
        
        chunk_q = q[c]
        chunk_t = t[c]
        
        keep = chunk_q[(chunk_t == labels_of_interest).any(axis=1)]
        
        proc_chunk.append(keep)
        
        if i % 100 == 0:
            pbar.update(100)
            
        i += 1
        


In [ ]:
idx_select = np.unique(np.concatenate(proc_chunk))

In [ ]:
len(idx_select) # this many regions have at least one of the labels in labels_of_interest

In [ ]:
# IMPORTANT: always close the hdf5 file after use

f.close()